In [1]:
! pip install datasets transformers seqeval evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.6 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=7347c7c93eed9d290290fc0b27ffbb7c2fba6334564753c850e88ae1dfd480a0
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.

In [2]:
from huggingface_hub import notebook_login

notebook_login()

# Fine-tuning a model on a token classification task-
The most common token classification tasks are-
* NER(Named-entity recognition)- Classify the entities in the text(Person,organization,location)
* POS(Part-of-speech tagging)- Grammatically classify the tokens(nouns,verb,adjective)
* Chunk(Chunking)- Grammatically classify the tokens and the group them into 'chunk' that go together.

In [3]:
# Load the Dataset
from datasets import load_dataset
task='ner'
model_checkpoint='distilbert-base-uncased'
batch_size=16

datasets=load_dataset('conll2003')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [4]:
datasets['train'].features[f"ner_tags"]

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

So the NER tags,there are four labels for NER here,each prefixed with "B"-for begining or "I"-for intermediate, that indicate if the token is the first one for the current gorup with the label or not
* "PER" for person
* "ORG" for organization
* "LOC" for location
* "MISC" for miscellaneous

In [5]:
label_list=datasets['train'].features[f"{task}_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [6]:
# Pre-processing the data by using Tokenizer
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
import transformers
assert isinstance(tokenizer,transformers.PreTrainedTokenizerFast)
# This ensures that our tokenizer is a fast tokenizer.

In [8]:
# Example
example=datasets['train'][4]
print(example['tokens'])

['Germany', "'s", 'representative', 'to', 'the', 'European', 'Union', "'s", 'veterinary', 'committee', 'Werner', 'Zwingmann', 'said', 'on', 'Wednesday', 'consumers', 'should', 'buy', 'sheepmeat', 'from', 'countries', 'other', 'than', 'Britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.']


In [9]:
tokenized_input=tokenizer(example['tokens'],is_split_into_words=True)
tokens=tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'])
print(tokens)

['[CLS]', 'germany', "'", 's', 'representative', 'to', 'the', 'european', 'union', "'", 's', 'veterinary', 'committee', 'werner', 'z', '##wing', '##mann', 'said', 'on', 'wednesday', 'consumers', 'should', 'buy', 'sheep', '##me', '##at', 'from', 'countries', 'other', 'than', 'britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.', '[SEP]']


In [10]:
len(example[f"{task}_tags"]),len(tokenized_input['input_ids'])

(31, 39)

In [11]:
print(tokenized_input.word_ids())

[None, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 10, 11, 11, 11, 12, 13, 14, 15, 16, 17, 18, 18, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, None]


In [12]:
word_ids=tokenized_input.word_ids()
aligned_labels=[-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels),len(tokenized_input['input_ids']))

39 39


In [13]:
label_all_tokens=True

In [14]:
# Will apply tokenizer function
def tokenize_and_align_labels(examples):
  tokenized_inputs=tokenizer(examples['tokens'],truncation=True,is_split_into_words=True)

  labels=[]
  for i,label in enumerate(examples[f"{task}_tags"]):
    word_ids=tokenized_inputs.word_ids(batch_index=i)
    previous_word_idx=None
    label_ids=[]
    for word_idx in word_ids:
      # Special tokens have a word id that is None. We set the label to -100 so they are automatically ignored in the loss function
      if word_idx is None:
        label_ids.append(-100)
      # We set the label for the first token of each word
      elif word_idx != previous_word_idx:
        label_ids.append(label[word_idx])
      else:
        label_ids.append(label[word_idx] if label_all_tokens else -100)
      previous_word_idx = word_idx
    labels.append(label_ids)
  tokenized_inputs['labels']=labels
  return tokenized_inputs


In [15]:
tokenize_and_align_labels(datasets['train'][:5])

{'input_ids': [[101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102], [101, 2848, 13934, 102], [101, 9371, 2727, 1011, 5511, 1011, 2570, 102], [101, 1996, 2647, 3222, 2056, 2006, 9432, 2009, 18335, 2007, 2446, 6040, 2000, 10390, 2000, 18454, 2078, 2329, 12559, 2127, 6529, 5646, 3251, 5506, 11190, 4295, 2064, 2022, 11860, 2000, 8351, 1012, 102], [101, 2762, 1005, 1055, 4387, 2000, 1996, 2647, 2586, 1005, 1055, 15651, 2837, 14121, 1062, 9328, 5804, 2056, 2006, 9317, 10390, 2323, 4965, 8351, 4168, 4017, 2013, 3032, 2060, 2084, 3725, 2127, 1996, 4045, 6040, 2001, 24509, 1012, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100], [-100, 1, 2, -100], [-100, 5, 0, 

In [16]:
# Will apply on entire dataset
tokenized_datasets=  datasets.map(tokenize_and_align_labels,batched=True)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

# Fine-Tune the model

In [17]:
from transformers import AutoModelForTokenClassification, TrainingArguments,Trainer
model= AutoModelForTokenClassification.from_pretrained(model_checkpoint,num_labels=len(label_list))

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
# Will apply training Arguments on the process
model_name=model_checkpoint.split("/")[-1]
args=TrainingArguments(
    f"{model_name}-finetuned-{task}",
    eval_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True
)

In [19]:
# So we are using data collator that will batch our processed examples together while applying padding to make them all same size
from transformers import DataCollatorForTokenClassification
data_collator=DataCollatorForTokenClassification(tokenizer)

In [20]:
# So we will load "seqeval" evaluation metric used for sequence labeling tasks.
import evaluate
metric=evaluate.load('seqeval')

In [21]:
labels=[label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels],references=[labels])

{'LOC': {'precision': np.float64(1.0),
  'recall': np.float64(1.0),
  'f1': np.float64(1.0),
  'number': np.int64(2)},
 'ORG': {'precision': np.float64(1.0),
  'recall': np.float64(1.0),
  'f1': np.float64(1.0),
  'number': np.int64(1)},
 'PER': {'precision': np.float64(1.0),
  'recall': np.float64(1.0),
  'f1': np.float64(1.0),
  'number': np.int64(1)},
 'overall_precision': np.float64(1.0),
 'overall_recall': np.float64(1.0),
 'overall_f1': np.float64(1.0),
 'overall_accuracy': 1.0}

So we will need to do a bit of post-processing on our predictions:
- select the predicted index (with the maximum logit) for each token
- convert it to its string label
- ignore everywhere we set a label of -100

The following function does all this post-processing on the result of `Trainer.evaluate` (which is a namedtuple containing predictions and labels) before applying the metric:

In [24]:
import numpy as np

def compute_metric(p):
  predictions,labels=p
  predictions=np.argmax(predictions,axis=2)
  # Remove ignored index (special tokens)
  true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
  true_labels = [
      [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
      for prediction, label in zip(predictions, labels)
  ]

  results = metric.compute(predictions=true_predictions, references=true_labels)
  return {
      "precision": results["overall_precision"],
      "recall": results["overall_recall"],
      "f1": results["overall_f1"],
      "accuracy": results["overall_accuracy"],
  }

In [27]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metric
)

<ipython-input-27-a17cdf91d1be>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [28]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ombhandwalkar38126 (ombhandwalkar38126-student) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.243700,0.072070,0.900142,0.921691,0.910789,0.979745
2,0.052400,0.060778,0.921128,0.935451,0.928234,0.983176
3,0.030500,0.060940,0.926339,0.938360,0.932311,0.983780


TrainOutput(global_step=2634, training_loss=0.08640983055827287, metrics={'train_runtime': 301.9149, 'train_samples_per_second': 139.519, 'train_steps_per_second': 8.724, 'total_flos': 510122266253334.0, 'train_loss': 0.08640983055827287, 'epoch': 3.0})

In [29]:
trainer.evaluate()

{'eval_loss': 0.06094033643603325,
 'eval_precision': 0.9263390392048592,
 'eval_recall': 0.9383599955252265,
 'eval_f1': 0.9323107702567521,
 'eval_accuracy': 0.9837800054013695,
 'eval_runtime': 5.6485,
 'eval_samples_per_second': 575.377,
 'eval_steps_per_second': 36.116,
 'epoch': 3.0}

In [30]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'LOC': {'precision': np.float64(0.9578427649069502),
  'recall': np.float64(0.9633307868601986),
  'f1': np.float64(0.9605789373452674),
  'number': np.int64(2618)},
 'MISC': {'precision': np.float64(0.8080251770259638),
  'recall': np.float64(0.834281072298944),
  'f1': np.float64(0.820943245403677),
  'number': np.int64(1231)},
 'ORG': {'precision': np.float64(0.8865784499054821),
  'recall': np.float64(0.9124513618677043),
  'f1': np.float64(0.8993288590604026),
  'number': np.int64(2056)},
 'PER': {'precision': np.float64(0.9762767710049424),
  'recall': np.float64(0.9765985497692815),
  'f1': np.float64(0.9764376338770804),
  'number': np.int64(3034)},
 'overall_precision': np.float64(0.9263390392048592),
 'overall_recall': np.float64(0.9383599955252265),
 'overall_f1': np.float64(0.9323107702567521),
 'overall_accuracy': 0.9837800054013695}

In [31]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1746096310.67f58ec0db17.715.0:   0%|          | 0.00/8.12k [00:00<?, ?B/s]

events.out.tfevents.1746096625.67f58ec0db17.715.1:   0%|          | 0.00/560 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/OmBhandwalkar/distilbert-base-uncased-finetuned-ner/commit/bab583c041bd100e787e836feb6d50a3a54f574f', commit_message='End of training', commit_description='', oid='bab583c041bd100e787e836feb6d50a3a54f574f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/OmBhandwalkar/distilbert-base-uncased-finetuned-ner', endpoint='https://huggingface.co', repo_type='model', repo_id='OmBhandwalkar/distilbert-base-uncased-finetuned-ner'), pr_revision=None, pr_num=None)